## READ FILE


[  0 820 422 544 132   0 595 865 273 166 356 955   0 773 367 749 991  85
 167   0 711 753 939 665 158   0 408 600 898 172 855 931 421 448 560 342
 759 843 135   0 652 316 742 240 741 585 664 588 501   6 310 198 714 341
 755 530 418   0  87 530 287   0 905  69   0 315 908   0 199 459 677 207
 738 484 868 380 359 320 186 902 378 336 750 619 771 330 850 948 779 736
 937 145 195 344 900 360 372 931 117 450   0   0 376 653 918]
104
918


In [49]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import pandas as pd
import numpy as np

# Define file path and sheet names
file_path = "DATA_corrected.xlsx"
sheet_list = ["6-periods (1)", "6-periods (2)", "12-periods (1)", "12-periods (2)", 
              "24-periods (1)", "24-periods (2)", "52-periods (1)", "52-periods (2)", 
              "104-periods (1)", "104-periods (2)"]

# DataFrame to store all results
results_df = pd.DataFrame(columns=["Sheet", "Period", "y", "x", "S", "b"])

# Process each sheet
for sheet_name in sheet_list:
    print(f"\nProcessing sheet: {sheet_name}")

    # Read the Excel file from the specific sheet
    df = pd.read_excel(file_path, sheet_name=sheet_name)
    
    # Insert dummy period 0
    demand_forecast = np.insert(df["Demand Forecast"].to_numpy(), 0, 0)
    setup_cost = np.insert(df["Setup Cost"].to_numpy(), 0, 0)
    production_cost = np.insert(df["Production cost"].to_numpy(), 0, 0)
    holding_cost = np.insert(df["Holding cost"].to_numpy(), 0, 0)
    backlogging_cost = np.insert(df["Backlogging cost"].to_numpy(), 0, 0)

    T = df.shape[0]
    print(f"Number of periods: {T}")

    # Create Gurobi model
    model = gp.Model("Uncapacitated Lot-Sizing Problem")

    # Decision variables
    y = model.addVars(T+1, vtype=GRB.BINARY, name="y")  
    x = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="x")  
    S = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="S")  
    b = model.addVars(T+1, vtype=GRB.CONTINUOUS, lb=0, name="b")  # Amount backlogged at the end of period t

    # Objective function
    model.setObjective(
            quicksum(setup_cost[t] * y[t] for t in range(1, T+1)) +
            quicksum(production_cost[t] * x[t] for t in range(1, T+1)) +
            quicksum(holding_cost[t] * S[t] for t in range(1, T+1)) +
            quicksum(backlogging_cost[t] * b[t] for t in range(1, T+1)),
            GRB.MINIMIZE
        )

    # Constraints
    model.addConstr(S[0] == 0, name="no_inventory0")
    model.addConstr(S[T] == 0, name="no_inventoryT")
    model.addConstr(b[0] == 0, name="no_backlogging0")
    model.addConstr(b[T] == 0, name="no_backloggingT")

    for t in range(1, T+1):
        model.addConstr(x[t] + S[t-1] - b[t-1] == demand_forecast[t] + S[t] - b[t], name=f"demand_satisfied_{t}")
        model.addConstr(x[t] <= (quicksum(demand_forecast[m] for m in range(t, T+1)) + b[t]) * y[t], name=f"setup_constraint_{t}")

    # Solve model
    model.optimize()

    # Check if the model found an optimal solution
    if model.status == GRB.OPTIMAL:
        print("\nOptimal solution found:")
        # Create a temporary DataFrame for the current sheet results
        sheet_results = pd.DataFrame(columns=["Sheet", "Period", "y", "x", "S", "b"])
        for t in range(1, T+1):
            # Store the results for the current sheet
            sheet_results = pd.concat([sheet_results, pd.DataFrame([{
                "Sheet": sheet_name,
                "Period": t,
                "y": y[t].X,
                "x": x[t].X,
                "S": S[t].X,
                "b": b[t].X
            }])], ignore_index=True)
        
        # Concatenate the results for all sheets
        results_df = pd.concat([results_df, sheet_results], ignore_index=True)

    else:
        print(f"No optimal solution found for sheet {sheet_name}.")

# Save results to a CSV file
results_df.to_csv("optimization_results.csv", index=False)
print("\nResults have been saved to 'optimization_results.csv'.")



Processing sheet: 6-periods (1)
Number of periods: 6
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 10 rows, 28 columns and 34 nonzeros
Model fingerprint: 0xf02f4325
Model has 6 quadratic constraints
Variable types: 21 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 2e+03]
  Objective range  [3e+00, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+02, 6e+02]
Presolve removed 4 rows and 7 columns
Presolve time: 0.00s
Presolved: 21 rows, 36 columns, 64 nonzeros
Presolved model has 10 SOS constraint(s)
Variable types: 25 continuous, 11 integer (11 binary)

Root relaxation: objective 3.499618e+04, 15 iterations, 0.00 seconds (0.00 work units)


C:\Users\ricca\AppData\Local\Temp\ipykernel_14652\131815249.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sheet_results = pd.concat([sheet_results, pd.DataFrame([{
C:\Users\ricca\AppData\Local\Temp\ipykernel_14652\131815249.py:80: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, sheet_results], ignore_index=True)



Optimal solution found:

Processing sheet: 12-periods (1)
Number of periods: 12
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 16 rows, 52 columns and 64 nonzeros
Model fingerprint: 0xbee7b666
Model has 12 quadratic constraints
Variable types: 39 continuous, 13 integer (13 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 5e+03]
  Objective range  [1e+00, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 9e+02]
Presolve removed 4 rows and 7 columns


C:\Users\ricca\AppData\Local\Temp\ipykernel_14652\131815249.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sheet_results = pd.concat([sheet_results, pd.DataFrame([{


Presolve time: 0.01s
Presolved: 45 rows, 78 columns, 142 nonzeros
Presolved model has 22 SOS constraint(s)
Variable types: 55 continuous, 23 integer (23 binary)

Root relaxation: objective 4.006714e+04, 45 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 40067.1352    0   17          - 40067.1352      -     -    0s
H    0     0                    70618.000000 46366.3426  34.3%     -    0s
     0     0 46366.3426    0   19 70618.0000 46366.3426  34.3%     -    0s
H    0     0                    67917.000000 47894.9579  29.5%     -    0s
     0     0 47894.9579    0   19 67917.0000 47894.9579  29.5%     -    0s
     0     0 48045.0753    0   20 67917.0000 48045.0753  29.3%     -    0s
     0     0 49380.3562    0   20 67917.0000 49380.3562  27.3%     -    0s
H    0     0                    55961.000000 49380.3562  11.8%     -    0s

...

C:\Users\ricca\AppData\Local\Temp\ipykernel_14652\131815249.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sheet_results = pd.concat([sheet_results, pd.DataFrame([{


Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 108 rows, 420 columns and 524 nonzeros
Model fingerprint: 0x56377807
Model has 104 quadratic constraints
Variable types: 315 continuous, 105 integer (105 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 5e+04]
  Objective range  [1e+00, 5e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 1e+03]
Presolve removed 5 rows and 9 columns
Presolve time: 0.00s
Presolved: 412 rows, 720 columns, 1335 nonzeros
Presolved model has 206 SOS constraint(s)
Variable types: 514 continuous, 206 integer (206 binary)

Root relaxation: objective 2.234785e+05, 433 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    

C:\Users\ricca\AppData\Local\Temp\ipykernel_14652\131815249.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sheet_results = pd.concat([sheet_results, pd.DataFrame([{


Number of periods: 104
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 108 rows, 420 columns and 524 nonzeros
Model fingerprint: 0xc0532a54
Model has 104 quadratic constraints
Variable types: 315 continuous, 105 integer (105 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 5e+04]
  Objective range  [1e+00, 5e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [6e+00, 1e+03]
Presolve removed 4 rows and 7 columns
Presolve time: 0.00s
Presolved: 413 rows, 722 columns, 1338 nonzeros
Presolved model has 206 SOS constraint(s)
Variable types: 515 continuous, 207 integer (207 binary)

Root relaxation: objective 4.080493e+05, 467 iterations, 0.01 seconds (0.00 work units)

    Nodes  

C:\Users\ricca\AppData\Local\Temp\ipykernel_14652\131815249.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sheet_results = pd.concat([sheet_results, pd.DataFrame([{


In [91]:
import matplotlib.pyplot as plt

# Generate and save histograms for each sheet
for sheet_name in results_df["Sheet"].unique():
    sheet_data = results_df[results_df["Sheet"] == sheet_name]

    plt.figure(figsize=(8, 5))
    plt.bar(sheet_data["Period"], sheet_data["y"], color='blue', alpha=0.7)
    plt.xlabel("Period")
    plt.ylabel("Inventory Level (S)")
    plt.title(f"Inventory Levels for {sheet_name}")
    plt.xticks(rotation=45)
    plt.grid(axis='y', linestyle='--', alpha=0.7)

    # Save histogram as an image
    hist_filename = f"histogram_{sheet_name.replace(' ', '_')}.png"
    plt.savefig(hist_filename)
    print(f"Histogram saved: {hist_filename}")
    plt.close()


Histogram saved: histogram_6-periods_(1).png
Histogram saved: histogram_6-periods_(2).png
Histogram saved: histogram_12-periods_(1).png
Histogram saved: histogram_12-periods_(2).png
Histogram saved: histogram_24-periods_(1).png
Histogram saved: histogram_24-periods_(2).png
Histogram saved: histogram_52-periods_(1).png
Histogram saved: histogram_52-periods_(2).png
Histogram saved: histogram_104-periods_(1).png
Histogram saved: histogram_104-periods_(2).png
